## this notebook is to extract all the investigation events
1. Step 1: extract all investigation instances
2. Step 2:extract the informationa and features during the investigation
   1. video sequence
   2. investigation event ID
   3. frame start
   4. frame end
   5. investigation duration
   6. paired attack start
   7. paired attack end
   8. com_x
   9. com_y
   10. linear_speed
   11. linear_acc
   12. rotation_speed
   13. rotation_acc
   14. label
3.  Step: clean and save the data

In [32]:
#import some packages
import numpy as np
import pandas as pd

In [1]:
#define some functions for feature extraction
def calc_displacement(x,y):
    """
    displacement: the x,y coordinate difference from the neighboring frames
    pad zero at the beginning
    """
    x_displacement = np.diff(x)
    y_displacement = np.diff(y)
    x_displacement = np.insert(x_displacement,0,0)
    y_displacement = np.insert(y_displacement,0,0)
    return x_displacement, y_displacement

def calc_velocity(x_displacement, y_displacement, dt):
    """
    calculate the velocity from the displacement and ft
    dt: time interval between two frames
    """
    vx = x_displacement / dt
    vy = y_displacement /dt
    return vx, vy

def calc_speed(vx, vy):
    """
    calculate the absolute value of velocity
    """
    speed = np.sqrt((vx**2+vy**2))
    return(speed)

def calc_acc(vx, vy, dt):
    """
    calculate the absolute value of acceleration from the velocity and dt
    dt: time interval between two frames
    """
    accx = np.insert(np.diff(vx),0,0)
    accy = np.insert(np.diff(vy),0,0)
    accx = accx /dt
    accy = accy/dt
    acc = np.sqrt(accx**2+accy**2)
    return (acc)

def head_position(self, earleft, earright):
    head_x = (earleft[:,0]+earright[:,0])/2
    head_y = (earleft[:,1]+earright[:,1])/2
    return head_x, head_y

def center_of_mass(self, keypoints):
    mouse_x = np.mean(keypoints[:,0,:], axis=1)
    mouse_y = np.mean(keypoints[:,1,:], axis=1)
    return mouse_x, mouse_y

def head_mouse_tail_angle(self, head, mouse, tail):
    v1 = head - mouse
    v2 = tail - mouse
    dot_product = np.sum(v1 * v2, axis=1)
    norm_v1 = np.linalg.norm(v1, axis=1)
    norm_v2 = np.linalg.norm(v2, axis=1)
    angle = np.arccos(np.clip(dot_product / (norm_v1 * norm_v2), -1.0, 1.0))
    return angle


def head_to_nose_vector(self, nose, head_x, head_y):
    head_to_nose_x = nose[:,0] - head_x
    head_to_nose_y = nose[:,1] - head_y
    return head_to_nose_x, head_to_nose_y

def head_direction_angle(self, head_to_nose_x, head_to_nose_y):
    head_direction_angle = np.arctan2(head_to_nose_y, head_to_nose_x) #the value range of arctan2 is [-pi,pi] compared to arctan[-pi/2, pi/2]
    return head_direction_angle

def intra_joint_distance(self, keypoints, joint1, joint2):
    return np.linalg.norm(keypoints[:,joint1,:]-keypoints[:,joint2,:], axis=1)

def intra_joint_angle(self, keypoints, joint1, joint2, joint3):
    v1 = keypoints[:,joint1,:]-keypoints[:,joint2,:]
    v2 = keypoints[:,joint3,:]-keypoints[:,joint2,:]
    dot_product = np.sum(v1 * v2, axis=1)
    norm_v1 = np.linalg.norm(v1, axis=1)
    norm_v2 = np.linalg.norm(v2, axis=1)
    # Handle cases where norm is zero to avoid division by zero
    norm_v1[norm_v1 == 0] = 1e-6
    norm_v2[norm_v2 == 0] = 1e-6
    angle = np.arccos(np.clip(dot_product / (norm_v1 * norm_v2), -1.0, 1.0)) # Clip to avoid floating point errors
    return angle


def inter_joint_distance(self, keypoints1, keypoints2, joint1, joint2):
    return np.linalg.norm(keypoints1[:,joint1,:]-keypoints2[:,joint2,:], axis=1)


def inter_joint_angle(self, keypoints1, keypoints2, joint1, joint2, joint3):
    # Angle between vector from joint2 (mouse2) to joint1 (mouse1) and vector from joint2 (mouse2) to joint3 (mouse2)
    v1 = keypoints1[:,joint1,:] - keypoints2[:,joint2,:]
    v2 = keypoints2[:,joint3,:] - keypoints2[:,joint2,:]
    dot_product = np.sum(v1 * v2, axis=1)
    norm_v1 = np.linalg.norm(v1, axis=1)
    norm_v2 = np.linalg.norm(v2, axis=1)
    # Handle cases where norm is zero to avoid division by zero
    norm_v1[norm_v1 == 0] = 1e-6
    norm_v2[norm_v2 == 0] = 1e-6
    angle = np.arccos(np.clip(dot_product / (norm_v1 * norm_v2), -1.0, 1.0))
    return angle

    

In [42]:
#define some functions for extracting investigations and paired investigation
#finding the I-A pair
#need a dic to store all -IA pair
#{"pair ID";"investigation_start";"investigation_end";"attack_start";"attack_end";"investigation_duration";"attack_intensity"}

#this function is to extract all start and end time of the investigation and attack behavior
def find_I_A_all(sequence):
    #initialize lists to store the start and end frame index of investigation and attack
    current_behavior = None
    current_start_frame = -1
    investigation_ID = 0 #start from 0
    attack_ID = 0
    for frame_idx, frame_behavior in enumerate(sequence['label']):
        #we use current_behavior as a flag, if the current behavior is not none(not the start), we decide whether it's investigation or attack
        if current_behavior is not None:
            if frame_behavior != current_behavior:
                #if the current frame is not the same behavior as the previous frame
                #we decide whether the previous bout is a investigation bout or an attack bout
                #and reset the flag
                if current_behavior == 1: #1 is investigation
                    sequence['end_frame'] = frame_idx - 1
                    sequence['duration'] = frame_idx - current_investigation_start_frame
                    sequence['investigation_ID'] = investigation_ID 
                    investigation_ID = investigation_ID + 1
                    current_behavior = None
                elif current_behavior == 0: #0 is attack
                    sequence['end_frame'] = frame_idx - 1
                    sequence['duration'] = frame_idx - current_attack_start_frame
                    sequence['attack_ID'] = attack_ID
                    atatck_ID = attack_ID + 1
                    current_behavior = None
                #if the current frame is stil the same behavior as the previous frame
                #we continue the looping
            elif current_behavior ==frame_behavior and current_behavior == 0:
                current_behavior = frame_behavior
            elif current_behavior ==frame_behavior and current_behavior == 1:
                current_behavior = frame_behavior

        #if we just finish a reset(a bout of investigation or attack), we decide whether starts a new investigation bout or attack bout
        if current_behavior is None and frame_behavior ==1:
            current_behavior = 1
            current_investigation_start_frame = frame_idx
            sequence['start_frame'] = current_investigation_start_frame
        elif current_behavior is None and frame_behavior ==0:
            current_behavior = 0
            current_attack_start_frame = frame_idx
            sequence['start_frame'] = current_attack_start_frame

    #deal with the last frame
    if frame_behavior == "investigation":
        sequence['end_frame'] = frame_idx
        sequence['duration'] = frame_idx - current_investigation_start_frame+1
        sequence['investigation_ID'] = investigation_ID 
        investigation_ID = investigation_ID + 1
    if frame_behavior == "attack":
        sequence['end_frame'] = frame_idx
        sequence['duration'] = frame_idx - current_attack_start_frame+1
        sequence['attack_ID'] = attack_ID
        atatck_ID = attack_ID + 1
    print(sequence)
    return(sequence)

def find_I_A_pair(sequence):    
    investigation = sequence[sequence['label']==1]
    attack = sequence[sequence['label']==0]
    investigation_start = investigation['start_frame'].unique()
    investigation_end = investigation['end_frame'].unique()
    attack_start =attack['start_frame'].unique()
    attack_end = attack['end_frame'].unique()

    #print("in")

    investigation_flag =-1
    attack_flag =0
    for i in np.arange(len(attack_start)):
        a_start = attack_start[i]
        a_end = attack_end[i]
        #set the flags

        # Find the most recent preceding investigation
        preceding_investigations = [(i_start, i_end) for i_start, i_end in zip(investigation_start, investigation_end) if i_end < a_start]
       # print(preceding_investigations)
        if preceding_investigations:
            # Get the last one
            #print(preceding_investigations)
            last_investigation_start, last_investigation_end = preceding_investigations[-1]
            if last_investigation_start !=investigation_flag:
                sequence[sequence['start_frame']==last_investigation_start]['paired'] = 1
                investigation_flag = last_investigation_start
            else:
                continue
        else:
            continue
    return(sequence) #[]

In [31]:
import os
# Check the current working directory
print("Current working directory:", os.getcwd())

Current working directory: g:\My Drive\100-PhD Study\110-Courses\2025Summer_Neuromatch\project\code


In [24]:
len(temp)

89

In [25]:
temp = list(sequence)
temp[0][1]['label']


507738    3
507739    1
507740    1
507741    1
507742    1
         ..
531543    3
531544    3
531545    3
531546    3
531547    3
Name: label, Length: 23810, dtype: int64

In [46]:
sequence_list = list(sequence)
updated_sequence_list = []
for i in range(len(sequence_list)):
    updated_sequence = find_I_A_all(sequence_list[i][1])
    updated_sequence2 = find_I_A_pair(updated_sequence)
    updated_sequence_list.append(updated_sequence2)

        Unnamed: 0                              sequence  frame  \
507738      507738  task1/test/mouse071_task1_annotator1      0   
507739      507739  task1/test/mouse071_task1_annotator1      1   
507740      507740  task1/test/mouse071_task1_annotator1      2   
507741      507741  task1/test/mouse071_task1_annotator1      3   
507742      507742  task1/test/mouse071_task1_annotator1      4   
...            ...                                   ...    ...   
531543      531543  task1/test/mouse071_task1_annotator1  23805   
531544      531544  task1/test/mouse071_task1_annotator1  23806   
531545      531545  task1/test/mouse071_task1_annotator1  23807   
531546      531546  task1/test/mouse071_task1_annotator1  23808   
531547      531547  task1/test/mouse071_task1_annotator1  23809   

                                               id  label  resident_x_nose  \
507738      task1/test/mouse071_task1_annotator10      3       720.980978   
507739      task1/test/mouse071_task1_ann

KeyError: 'end_frame'

In [44]:
updated_sequence

,Unnamed: 0,sequence,frame,id,label,resident_x_nose,resident_x_left_ear,resident_x_right_ear,resident_x_neck,resident_x_left_hip,...,intruder_x_right_hip,intruder_x_tail_base,intruder_y_nose,intruder_y_left_ear,intruder_y_right_ear,intruder_y_neck,intruder_y_left_hip,intruder_y_right_hip,intruder_y_tail_base,start_frame
228990,228990,task1/train/mouse036_task1_annotator1,0,task1/train/mouse036_task1_annotator10,3,204.757671,257.757671,261.757671,273.757671,313.757671,...,715.857178,653.857178,356.089869,398.089869,346.089869,372.089869,413.089869,329.089869,374.089869,39
228991,228991,task1/train/mouse036_task1_annotator1,1,task1/train/mouse036_task1_annotator11,3,218.363091,244.363091,278.363091,273.363091,299.363091,...,722.889380,667.889380,351.458355,396.458355,337.458355,368.458355,407.458355,336.458355,375.458355,39
228992,228992,task1/train/mouse036_task1_annotator1,2,task1/train/mouse036_task1_annotator12,3,225.117972,242.117972,272.117972,268.117972,297.117972,...,755.659058,705.659058,339.421355,387.421355,323.421355,361.421355,401.421355,338.421355,375.421355,39
228993,228993,task1/train/mouse036_task1_annotator1,3,task1/train/mouse036_task1_annotator13,3,220.884280,238.884280,268.884280,264.884280,295.884280,...,832.010803,771.010803,325.499951,378.499951,313.499951,353.499951,407.499951,330.499951,382.499951,39
228994,228994,task1/train/mouse036_task1_annotator1,4,task1/train/mouse036_task1_annotator14,3,206.673218,220.673218,263.673218,254.673218,289.673218,...,875.990540,835.990540,311.723130,366.723130,305.723130,343.723130,411.723130,331.723130,381.723130,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229081,229081,task1/train/mouse036_task1_annotator1,91,task1/train/mouse036_task1_annotator191,0,843.185443,820.185443,798.185443,795.185443,779.185443,...,852.185492,880.185492,260.339395,322.339395,338.339395,349.339395,401.339395,396.339395,438.339395,39
229082,229082,task1/train/mouse036_task1_annotator1,92,task1/train/mouse036_task1_annotator192,0,840.357166,815.357166,783.357166,797.357166,798.357166,...,847.120868,896.120868,211.705226,276.705226,304.705226,305.705226,369.705226,378.705226,422.705226,39
229083,229083,task1/train/mouse036_task1_annotator1,93,task1/train/mouse036_task1_annotator193,0,789.281067,797.281067,747.281067,766.281067,791.281067,...,847.129028,872.129028,176.800018,264.800018,221.800018,270.800018,362.800018,334.800018,414.800018,39
229084,229084,task1/train/mouse036_task1_annotator1,94,task1/train/mouse036_task1_annotator194,0,771.900574,776.900574,729.900574,752.900574,794.900574,...,933.706665,961.706665,203.002019,213.002019,265.002019,252.002019,278.002019,278.002019,219.002019,39


In [48]:
import numpy as np
import pandas as pd

# ---------- Helper feature functions ----------
def calc_displacement(x, y):
    x_disp = np.diff(x, prepend=x[0])
    y_disp = np.diff(y, prepend=y[0])
    return x_disp, y_disp

def calc_velocity(x_disp, y_disp, dt):
    return x_disp / dt, y_disp / dt

def calc_speed(vx, vy):
    return np.sqrt(vx**2 + vy**2)

def calc_acc(vx, vy, dt):
    accx = np.diff(vx, prepend=vx[0]) / dt
    accy = np.diff(vy, prepend=vy[0]) / dt
    return np.sqrt(accx**2 + accy**2)

def head_mouse_tail_angle(_, head, mouse, tail):
    v1 = head - mouse
    v2 = tail - mouse
    dot = np.sum(v1 * v2, axis=1)
    norm1 = np.linalg.norm(v1, axis=1)
    norm2 = np.linalg.norm(v2, axis=1)
    norm1[norm1 == 0] = 1e-6
    norm2[norm2 == 0] = 1e-6
    return np.arccos(np.clip(dot / (norm1 * norm2), -1.0, 1.0))

# ---------- Step 1: Extract all investigation/attack bouts ----------
def find_I_A_all(sequence):
    bouts = []
    current_behavior = None
    start_frame = None
    investigation_ID = 0
    attack_ID = 0

    for frame_idx, label in enumerate(sequence['label']):
        if current_behavior is None:
            current_behavior = label
            start_frame = frame_idx
        elif label != current_behavior:
            end_frame = frame_idx - 1
            duration = end_frame - start_frame + 1
            if current_behavior == 1:
                bouts.append({
                    'start_frame': start_frame,
                    'end_frame': end_frame,
                    'label': 1,
                    'investigation_ID': investigation_ID
                })
                investigation_ID += 1
            elif current_behavior == 0:
                bouts.append({
                    'start_frame': start_frame,
                    'end_frame': end_frame,
                    'label': 0,
                    'attack_ID': attack_ID
                })
                attack_ID += 1
            current_behavior = label
            start_frame = frame_idx

    # Handle last bout
    if current_behavior is not None:
        end_frame = len(sequence) - 1
        duration = end_frame - start_frame + 1
        if current_behavior == 1:
            bouts.append({
                'start_frame': start_frame,
                'end_frame': end_frame,
                'label': 1,
                'investigation_ID': investigation_ID
            })
        elif current_behavior == 0:
            bouts.append({
                'start_frame': start_frame,
                'end_frame': end_frame,
                'label': 0,
                'attack_ID': attack_ID
            })

    return pd.DataFrame(bouts)

# ---------- Step 2: Find which investigations are followed by attacks ----------
def find_I_A_pair(bout_df):
    investigation = bout_df[bout_df['label'] == 1].copy()
    attack = bout_df[bout_df['label'] == 0].copy()

    paired_flags = []
    for _, inv in investigation.iterrows():
        inv_end = inv['end_frame']
        # Check if any attack starts shortly after
        followed = any((attack['start_frame'] > inv_end) & (attack['start_frame'] <= inv_end + 15))  # 15-frame window
        paired_flags.append(int(followed))

    investigation['label'] = paired_flags  # overwrite 'label' to 1 if paired
    return investigation.reset_index(drop=True)

# ---------- Step 3: Extract features per investigation bout ----------
def extract_investigation_bouts_with_features(sequence_name, inv_bouts_df, keypoints, dt=1/30):
    features = []

    for _, row in inv_bouts_df.iterrows():
        s, e = row['start_frame'], row['end_frame']
        bout_kp = keypoints[s:e+1]

        # Center of mass
        mouse_x = np.mean(bout_kp[:, 0, :], axis=1)
        mouse_y = np.mean(bout_kp[:, 1, :], axis=1)

        x_disp, y_disp = calc_displacement(mouse_x, mouse_y)
        vx, vy = calc_velocity(x_disp, y_disp, dt)
        speed = calc_speed(vx, vy)
        acc = calc_acc(vx, vy, dt)

        head = (bout_kp[:, 1, :] + bout_kp[:, 2, :]) / 2
        tail = bout_kp[:, -1, :]
        body = np.mean(bout_kp, axis=1)
        angle = head_mouse_tail_angle(None, head, body, tail)
        angle_speed = np.diff(angle, prepend=angle[0]) / dt

        features.append({
            'sequence_name': sequence_name,
            'bout_ID': row['investigation_ID'],
            'start_frame': s,
            'end_frame': e,
            'label': row['label'],
            'mean_speed': np.mean(speed),
            'mean_acc': np.mean(acc),
            'mean_body_angle': np.mean(angle),
            'mean_angle_speed': np.mean(np.abs(angle_speed))
        })

    return features
